In [ ]:
import pandas as pd
import requests
import io
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import glob
from scipy import stats
import scienceplots

plt.style.use('science')

FOLDER = datetime.now().strftime('%Y%m%d')
ASSETS = "../assets/"+FOLDER+"/"
INPUT = "../input/"+FOLDER+"/"
OUTPUT = "../output/"+FOLDER+"/"

glob.os.makedirs(ASSETS, exist_ok=True)
glob.os.makedirs(INPUT, exist_ok=True)
glob.os.makedirs(OUTPUT, exist_ok=True)

LINE_COLOR='#1B4244'



In [ ]:
import requests, pandas as pd, time

BASE = "https://community-api.coinmetrics.io/v4/timeseries/asset-metrics"

def fetch_asset_metrics_json(assets, metrics, start_time, frequency="1d", page_size=10000):
    params = {
        "assets": ",".join(assets),
        "metrics": ",".join(metrics),
        "frequency": frequency,
        "start_time": start_time,
        "page_size": page_size,
        "format": "json",
    }
    url = BASE
    all_rows = []

    while url:
        r = requests.get(url, params=params if url == BASE else None, timeout=30)
        if r.status_code == 429:
            # community limit: back off and retry
            time.sleep(1.0)
            continue
        r.raise_for_status()
        data = r.json()
        all_rows.extend(data.get("data", []))
        # follow pagination
        url = data.get("next_page_url")
        params = None  # only pass params on first call

    return pd.DataFrame(all_rows)

In [ ]:
df_onchain = fetch_asset_metrics_json(
    assets=["btc"],
    metrics=["HashRate", "PriceUSD", "FeeTotUSD"],
    start_time="2009-01-03",
)

In [ ]:
for col in ["HashRate", "PriceUSD", "FeeTotUSD"]:
    if col in df_onchain:
        df_onchain[col] = pd.to_numeric(df_onchain[col], errors="coerce")

In [ ]:
df_onchain["timestamp"]=pd.to_datetime(df_onchain["time"]).dt.date
df_onchain["market_price"]=df_onchain["PriceUSD"]
df_onchain["hash_rate"]=df_onchain["HashRate"]
df_onchain["daily_transaction_fees"]=df_onchain["FeeTotUSD"]/df_onchain["PriceUSD"]

In [ ]:
df_onchain=df_onchain[["timestamp", "hash_rate","market_price", "daily_transaction_fees"]].fillna(0)

In [ ]:
with plt.style.context(['science','ieee']):
    plt.figure(figsize=(4, 2.25))
    plt.grid(True, linestyle='-', alpha=0.7, linewidth=0.2)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.plot(df_onchain["timestamp"], df_onchain["daily_transaction_fees"],color=LINE_COLOR, linewidth=0.4)
    plt.title('Daily Transaction Fees', fontsize=12)
    plt.ylabel('Fee (BTC/day)', fontsize=10)
    plt.legend()
    plt.savefig(ASSETS+'transactionfees1.png')
    plt.show()

In [ ]:
with plt.style.context(['science','ieee']):
    plt.figure(figsize=(4, 2.25))
    plt.grid(True, linestyle='-', alpha=0.7, linewidth=0.2)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.plot(df_onchain["timestamp"], df_onchain["hash_rate"],color=LINE_COLOR, linewidth=0.4)
    plt.title('Global Bitcoin Hashrate', fontsize=12)
    plt.ylabel('Hashrate (TH/s)', fontsize=10)
    plt.savefig(ASSETS+'hashrate1.png')
    plt.show()

In [ ]:
df_onchain.set_index("timestamp").to_csv(OUTPUT+'onchain.csv')

In [ ]:
import pandas as pd

def create_btc_time_series():
    # Define the data as lists
    years = [
        '2009-01-03', '2012-11-28', '2016-06-09', '2020-05-11', '2024-04-20', '2028-06-01', '2032-06-01', '2036-06-01', 
        '2040-06-01', '2044-06-01', '2048-06-01', '2052-06-01', '2056-06-01', '2060-06-01', '2064-06-01', '2068-06-01', 
        '2072-06-01', '2076-06-01', '2080-06-01', '2084-06-01', '2088-06-01', '2092-06-01', '2096-06-01', '2100-06-01', 
        '2104-06-01', '2108-06-01', '2112-06-01', '2116-06-01', '2120-06-01', '2124-06-01', '2128-06-01', '2132-06-01', 
        '2136-06-01', '2140-06-01'
    ]

    block_reward = [
        50, 25, 12.5, 6.25, 3.125, 1.5625, 0.78125, 0.390625, 0.1953125,
        0.09765625, 0.048828125, 0.024414063, 0.012207031, 0.006103516,
        0.003051758, 0.001525879, 0.000762939, 0.00038147, 0.000190735,
        0.000095367, 0.000047684, 0.000023842, 0.000011921, 0.00000596,
        0.00000298, 0.00000149, 0.00000075, 0.00000037, 0.00000019,
        0.00000009, 0.00000005, 0.00000002, 0.00000001, 0.00000001
    ]

    total_btc_mined = [
        0, 10500000, 15750000, 18375000, 19687500, 20343750, 20671875,
        20835937.5, 20917968.75, 20958984.375, 20979492.1875, 20989746.09375,
        20994873.046875, 20997436.5234375, 20998718.26171875, 20999359.130859375,
        20999679.565429688, 20999839.782714844, 20999919.891357422,
        20999959.945678711, 20999979.972839355, 20999989.986419678,
        20999994.993209839, 20999997.496604919, 20999998.74830246,
        20999999.37415123, 20999999.687075615, 20999999.843537807,
        20999999.921768904, 20999999.960884452, 20999999.980442226,
        20999999.990221113, 20999999.995110556, 21000000
    ]

    new_btc_mined = [
        0, 10500000, 5250000, 2625000, 1312500, 656250, 328125, 164062.5,
        82031.25, 41015.625, 20507.8125, 10253.90625, 5126.953125,
        2563.4765625, 1281.73828125, 640.869140625, 320.4345703125,
        160.21728515625, 80.108642578125, 40.0543212890625, 20.02716064453125,
        10.013580322265625, 5.0067901611328125, 2.5033950805664062,
        1.2516975402832031, 0.6258487701416016, 0.3129243850708008,
        0.1564621925354004, 0.0782310962677002, 0.0391155481338501,
        0.01955777406692505, 0.009778887033462525, 0.0048894435167312625,
        0.0048894435167312625
    ]

    pct_all_btc_mined = [
        "0.00%", "50.00%", "75.00%", "87.50%", "93.75%", "96.88%", "98.44%",
        "99.22%", "99.61%", "99.80%", "99.90%", "99.95%", "99.98%", "99.99%",
        "99.994%", "99.997%", "99.998%", "99.999%", "99.9996%", "99.9998%",
        "99.9999%", "99.99995%", "99.99998%", "99.99999%", "99.999994%",
        "99.999997%", "99.999999%", "99.9999993%", "99.9999996%",
        "99.9999998%", "99.9999999%", "99.99999995%", "99.99999998%", "100.00%"
    ]

    # Create a DataFrame from the data
    df = pd.DataFrame({
        'Year': years,
        'Block Reward': block_reward,
        'Total BTC Mined': total_btc_mined,
        'New BTC Mined': new_btc_mined,
        '% of All BTC Mined': pct_all_btc_mined
    })

    # Convert the 'Year' column to a datetime column representing January 1 of that year
    df['Date'] = pd.to_datetime(df['Year'])

    # Set the datetime column as the index and drop the 'Year' column if desired
    df.set_index('Date', inplace=True)
    df.drop(columns='Year', inplace=True)

    return df

In [ ]:
sim_btc = create_btc_time_series()
sim_btc

In [ ]:
sim_btc_res= sim_btc[['Total BTC Mined']].resample('D').interpolate(method='linear')

In [ ]:
sim_btc_res.rename(columns={'Total BTC Mined':'total_bitcoins'}).to_csv(OUTPUT+'simulated_bitcoins.csv')
sim_btc.rename(columns={'Total BTC Mined':'total_bitcoins'}).to_csv(OUTPUT+'simulated_bitcoins_lores.csv')

In [ ]:
df_supply = fetch_asset_metrics_json(
    assets=["btc"],
    metrics=["SplyCur"],
    start_time="2009-01-03",
)
df_supply["SplyCur"] = pd.to_numeric(df_supply["SplyCur"], errors="coerce")

In [ ]:
df_supply["timestamp"]=pd.to_datetime(df_supply["time"]).dt.date
df_supply["total_bitcoins"]=df_supply["SplyCur"]

In [ ]:
df_supply = df_supply[["timestamp", "total_bitcoins"]]
df_supply.set_index('timestamp', inplace=True)

In [ ]:
with plt.style.context(['science','ieee']):
    plt.figure(figsize=(4, 3))
    plt.plot(df_supply.index, df_supply['total_bitcoins'], label='Mined Bitcoins', linewidth=0.4, color=LINE_COLOR)
    plt.plot(sim_btc_res.index, sim_btc_res['Total BTC Mined'], 'r-', label='Simulated Mined Bitcoins' , linewidth=0.4)
    plt.vlines(sim_btc.index,0,sim_btc['Total BTC Mined'], colors='b', linestyles='dashed', label='Halving Events', linewidth=0.2)

    plt.title('Mined Bitcoins', fontsize=12)
    plt.ylabel('Bitcoins (BTC)', fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend()
    # Rotate x-axis labels for better readability
    plt.xticks(rotation=45)

    # Add some padding to prevent label cutoff
    plt.tight_layout()
    plt.show()

In [ ]:
future_bitcoins = sim_btc_res[df_supply.index[-1]:][['Total BTC Mined']].rename(columns={'Total BTC Mined': 'total_bitcoins'})

In [ ]:
delta = df_supply["total_bitcoins"][-1]-future_bitcoins['total_bitcoins'][0]
df_supply["total_bitcoins"][-1],future_bitcoins['total_bitcoins'][0]

In [ ]:
future_bitcoins['total_bitcoins'] = future_bitcoins['total_bitcoins']+delta

In [ ]:
df_supply["total_bitcoins"][-1],future_bitcoins['total_bitcoins'][0]

In [ ]:
df_supply = pd.concat([df_supply, future_bitcoins[1:]], axis=0)

In [ ]:
difff=sim_btc_res.diff()

In [ ]:
with plt.style.context(['science','ieee']):
    plt.figure(figsize=(4, 2.25))
    plt.grid(True, linestyle='-', alpha=0.7, linewidth=0.2)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.plot(difff.index[2:], difff['Total BTC Mined'][2:], color=LINE_COLOR, linewidth=0.4)
    plt.ylabel('Reward (BTC)')
    plt.title('Daily Block Rewards')
    plt.savefig(ASSETS+'blockrewards1.png')
    plt.show()

In [ ]:
with plt.style.context(['science','ieee']):
    plt.figure(figsize=(4, 2.25))
    plt.grid(True, linestyle='-', alpha=0.7, linewidth=0.2)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.plot(df_supply.index, df_supply['total_bitcoins'],color=LINE_COLOR, linewidth=0.4, label='Mined Bitcoins')
    plt.vlines(sim_btc.index,0,sim_btc['Total BTC Mined'], colors='b', linestyles='dashed', label='Halving Events', linewidth=0.2)
    plt.title('Bitcoins in Circulation', fontsize=12)
    plt.ylabel('Bitcoins (BTC)', fontsize=10)
    plt.legend()
    plt.savefig(ASSETS+'minedbitcoin1.png')
    plt.show()

In [ ]:
df_supply.reset_index(inplace=True)
df_supply["index"]=pd.to_datetime(df_supply["index"]).dt.date
df_supply.set_index('index', inplace=True)

In [ ]:
df_supply[["total_bitcoins"]].to_csv(OUTPUT+'total_bitcoins.csv')